# TP 3 : Détection de sentiments avec un RNN - Mehdi ATTAOUI

In [53]:
import numpy as np
import pandas as pd 

## 1. Données d’apprentissage : 

In [54]:
phrases = [
    "Je suis très content",
    "C'était une belle journée",
    "Je suis déçu",
    "C'était horrible",
    "J'adore ce film",
    "Je déteste ce livre"
]

sentiment = [1, 1, 0, 0, 1, 0]  
# Step 2: Create the DataFrame
data = pd.DataFrame({
    'Phrase': phrases,
    'Sentiment': sentiment
})


In [55]:
data

,Phrase,Sentiment
0,Je suis très content,1
1,C'était une belle journée,1
2,Je suis déçu,0
3,C'était horrible,0
4,J'adore ce film,1
5,Je déteste ce livre,0


## 2. Prétraitement

### importation de NLTK

In [56]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mehdi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### tokenization avec expressions regulier pour gerer les ponctuations du langue francais

In [57]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+")

data['Tokens'] = data['Phrase'].apply(lambda x: tokenizer.tokenize(x.lower()))

data

,Phrase,Sentiment,Tokens
0,Je suis très content,1,"[je, suis, très, content]"
1,C'était une belle journée,1,"[c, était, une, belle, journée]"
2,Je suis déçu,0,"[je, suis, déçu]"
3,C'était horrible,0,"[c, était, horrible]"
4,J'adore ce film,1,"[j, adore, ce, film]"
5,Je déteste ce livre,0,"[je, déteste, ce, livre]"


### creation du vocabulaire, et convertir en sequence des indices

In [58]:
tokens = data['Tokens'].explode()
vocab_list = sorted(list(set(tokens)))
word_to_index = {"<PAD>": 0}
word_to_index.update({word: idx + 1 for idx, word in enumerate(vocab_list)})

# Conversion en indices
data['Token_Indices'] = data['Tokens'].apply(lambda tokens: [word_to_index[word] for word in tokens])

In [59]:
data

,Phrase,Sentiment,Tokens,Token_Indices
0,Je suis très content,1,"[je, suis, très, content]","[11, 14, 15, 5]"
1,C'était une belle journée,1,"[c, était, une, belle, journée]","[3, 17, 16, 2, 12]"
2,Je suis déçu,0,"[je, suis, déçu]","[11, 14, 7]"
3,C'était horrible,0,"[c, était, horrible]","[3, 17, 9]"
4,J'adore ce film,1,"[j, adore, ce, film]","[10, 1, 4, 8]"
5,Je déteste ce livre,0,"[je, déteste, ce, livre]","[11, 6, 4, 13]"


### ajoutons une jeton pad = 0 

In [60]:
def pad_sequences_numpy(sequences, max_len=None, pad_value=0):
    if max_len is None:
        max_len = max(len(seq) for seq in sequences)
    padded = np.full((len(sequences), max_len), pad_value)
    for i, seq in enumerate(sequences):
        padded[i, :len(seq)] = seq
    return padded

padded_np = pad_sequences_numpy(data['Token_Indices'].tolist(), pad_value=word_to_index["<PAD>"])
data['Padded_Indices'] = list(padded_np)

data

,Phrase,Sentiment,Tokens,Token_Indices,Padded_Indices
0,Je suis très content,1,"[je, suis, très, content]","[11, 14, 15, 5]","[11, 14, 15, 5, 0]"
1,C'était une belle journée,1,"[c, était, une, belle, journée]","[3, 17, 16, 2, 12]","[3, 17, 16, 2, 12]"
2,Je suis déçu,0,"[je, suis, déçu]","[11, 14, 7]","[11, 14, 7, 0, 0]"
3,C'était horrible,0,"[c, était, horrible]","[3, 17, 9]","[3, 17, 9, 0, 0]"
4,J'adore ce film,1,"[j, adore, ce, film]","[10, 1, 4, 8]","[10, 1, 4, 8, 0]"
5,Je déteste ce livre,0,"[je, déteste, ce, livre]","[11, 6, 4, 13]","[11, 6, 4, 13, 0]"


## creation RNN simple

In [ ]:
import torch
import torch.nn as nn

class SimpleRNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, rnn_type="GRU"):
        super(SimpleRNNClassifier, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        
        if rnn_type == "LSTM":
            self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        else:
            self.rnn = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)  
        output, hidden = self.rnn(embedded) 
        
        if isinstance(hidden, tuple): 
            hidden = hidden[0]

        hidden = hidden.squeeze(0)  
        out = self.fc(hidden)       
        out = self.sigmoid(out)     
        return out


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Convertir les données en tenseurs PyTorch
X = torch.tensor(data["Padded_Indices"], dtype=torch.long)
y = torch.tensor(data['Sentiment'].values, dtype=torch.float32).unsqueeze(1)  

# Créer DataLoader
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

### Fonction Cout

In [63]:
criterion = nn.BCELoss()  

In [ ]:
import torch.optim as optim
import matplotlib.pyplot as plt

# Recréons le modèle (au cas où)
model = SimpleRNNClassifier(vocab_size=len(word_to_index), embedding_dim=16, hidden_dim=32, rnn_type="LSTM")

# Optimiseur
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Pour stocker les pertes
losses = []

# Entraînement
num_epochs = 200
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch_X, batch_y in loader:
        optimizer.zero_grad()
        outputs = model(batch_X)               # prédictions
        loss = criterion(outputs, batch_y)     # calcul de la perte
        loss.backward()                        # rétropropagation
        optimizer.step()                       # mise à jour des poids
        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    losses.append(avg_loss)
    print(f"Époque {epoch+1}/{num_epochs} - Perte moyenne : {avg_loss:.4f}")


Époque 1/200 - Perte moyenne : 0.7149
Époque 2/200 - Perte moyenne : 0.6776
Époque 3/200 - Perte moyenne : 0.6531
Époque 4/200 - Perte moyenne : 0.6247
Époque 5/200 - Perte moyenne : 0.5661
Époque 6/200 - Perte moyenne : 0.4978
Époque 7/200 - Perte moyenne : 0.3835
Époque 8/200 - Perte moyenne : 0.2458
Époque 9/200 - Perte moyenne : 0.1203
Époque 10/200 - Perte moyenne : 0.0614
Époque 11/200 - Perte moyenne : 0.0257
Époque 12/200 - Perte moyenne : 0.0126
Époque 13/200 - Perte moyenne : 0.0063
Époque 14/200 - Perte moyenne : 0.0036
Époque 15/200 - Perte moyenne : 0.0023
Époque 16/200 - Perte moyenne : 0.0017
Époque 17/200 - Perte moyenne : 0.0014
Époque 18/200 - Perte moyenne : 0.0011
Époque 19/200 - Perte moyenne : 0.0010
Époque 20/200 - Perte moyenne : 0.0009
Époque 21/200 - Perte moyenne : 0.0008
Époque 22/200 - Perte moyenne : 0.0007
Époque 23/200 - Perte moyenne : 0.0007
Époque 24/200 - Perte moyenne : 0.0006
Époque 25/200 - Perte moyenne : 0.0006
Époque 26/200 - Perte moyenne : 0.

### Fonction predection

In [ ]:
def predict_sentiment(phrase, model, tokenizer, word_to_index, max_len=None):
    model.eval()
    with torch.no_grad():
       
        tokens = tokenizer.tokenize(phrase.lower())
        
    
        indices = [word_to_index.get(word, 0) for word in tokens]
        
      
        if max_len is None:
            max_len = X.shape[1]
        padded = indices + [0]*(max_len - len(indices))
        padded = padded[:max_len]  
        # 4. Convertir en tenseur
        input_tensor = torch.tensor([padded], dtype=torch.long) 
        # 5. Prédiction
        output = model(input_tensor)
        prob = output.item()

        # 6. Interprétation
        label = 1 if prob >= 0.5 else 0
        return prob, label


In [66]:
phrase1 = "je suis heureux"
phrase2 = "je suis triste"

prob1, label1 = predict_sentiment(phrase1, model, tokenizer, word_to_index)
prob2, label2 = predict_sentiment(phrase2, model, tokenizer, word_to_index)

print(f"Phrase: '{phrase1}' → Sentiment prédit: {label1} (proba={prob1:.4f})")
print(f"Phrase: '{phrase2}' → Sentiment prédit: {label2} (proba={prob2:.4f})")


Phrase: 'je suis heureux' → Sentiment prédit: 0 (proba=0.0097)
Phrase: 'je suis triste' → Sentiment prédit: 0 (proba=0.0097)


### ajouter heureux et triste pour que notre model peut l'apprendre

In [ ]:

new_phrases = [
    "Je suis heureux",     
    "Je suis triste"       
]
new_sentiments = [1, 0]


new_data = pd.DataFrame({
    'Phrase': new_phrases,
    'Sentiment': new_sentiments
})
data_extended = pd.concat([data[['Phrase', 'Sentiment']], new_data], ignore_index=True)


data_extended['Tokens'] = data_extended['Phrase'].apply(lambda x: tokenizer.tokenize(x.lower()))


tokens = data_extended['Tokens'].explode()
vocab_list = sorted(list(set(tokens)))
word_to_index = {"<PAD>": 0}
word_to_index.update({word: idx + 1 for idx, word in enumerate(vocab_list)})

data_extended['Token_Indices'] = data_extended['Tokens'].apply(lambda tokens: [word_to_index[word] for word in tokens])

padded_np = pad_sequences_numpy(data_extended['Token_Indices'].tolist(), pad_value=word_to_index["<PAD>"])
data_extended['Padded_Indices'] = list(padded_np)

X_ext = torch.tensor(data_extended['Padded_Indices'].tolist(), dtype=torch.long)
y_ext = torch.tensor(data_extended['Sentiment'].values, dtype=torch.float32).unsqueeze(1)

# 🔹 DataLoaer
dataset_ext = TensorDataset(X_ext, y_ext)
loader_ext = DataLoader(dataset_ext, batch_size=2, shuffle=True)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

X = torch.tensor(data["Padded_Indices"], dtype=torch.long)
y = torch.tensor(data['Sentiment'].values, dtype=torch.float32).unsqueeze(1)


dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:

model = SimpleRNNClassifier(vocab_size=len(word_to_index), embedding_dim=16, hidden_dim=32, rnn_type="LSTM")
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()


losses = []
num_epochs = 200
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch_X, batch_y in loader_ext:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(loader_ext)
    losses.append(avg_loss)
    if epoch % 20 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")


Epoch 1/200 - Loss: 0.6984
Epoch 21/200 - Loss: 0.0004
Epoch 41/200 - Loss: 0.0002
Epoch 61/200 - Loss: 0.0001
Epoch 81/200 - Loss: 0.0001
Epoch 101/200 - Loss: 0.0001
Epoch 121/200 - Loss: 0.0001
Epoch 141/200 - Loss: 0.0000
Epoch 161/200 - Loss: 0.0000
Epoch 181/200 - Loss: 0.0000


In [ ]:
model = SimpleRNNClassifier(vocab_size=len(word_to_index), embedding_dim=16, hidden_dim=32, rnn_type="LSTM")
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

losses = []
num_epochs = 200
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch_X, batch_y in loader_ext:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(loader_ext)
    losses.append(avg_loss)
    if epoch % 20 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")


Epoch 1/200 - Loss: 0.7067
Epoch 21/200 - Loss: 0.0005
Epoch 41/200 - Loss: 0.0003
Epoch 61/200 - Loss: 0.0002
Epoch 81/200 - Loss: 0.0001
Epoch 101/200 - Loss: 0.0001
Epoch 121/200 - Loss: 0.0001
Epoch 141/200 - Loss: 0.0001
Epoch 161/200 - Loss: 0.0000
Epoch 181/200 - Loss: 0.0000


### on remarque que le model peut detecter heureux et triste 

In [ ]:
phrase1 = "je suis heureux"
phrase2 = "je suis triste"

prob1, label1 = predict_sentiment(phrase1, model, tokenizer, word_to_index)
prob2, label2 = predict_sentiment(phrase2, model, tokenizer, word_to_index)

print(f"Phrase: '{phrase1}' → Sentiment prédit: {label1} (proba={prob1:.4f})")
print(f"Phrase: '{phrase2}' → Sentiment prédit: {label2} (proba={prob2:.4f})")


Phrase: 'je suis heureux' → Sentiment prédit: 1 (proba=1.0000)
Phrase: 'je suis triste' → Sentiment prédit: 0 (proba=0.0000)


# pourquoi ajoutons jetons <pad>?

#### Parce que les phrases n’ont pas toutes la même longueur, mais le modèle a besoin que tout soit de la même taille pour apprendre correctement. Alors on ajoute des <PAD> pour compléter les phrases courtes

# 1 Pourquoi un RNN est utile ici ?


#### Parce qu’un RNN gere bien les sequences, on sait bien que le text est tous lie entre eux , les mots ne sont pas indepandants, donc on veux un reseaux neuronnes qui garde l'antecedant pour serve le prochain , ce que RNN peux faire ,et un reseaux neuronnes normal NN va echouer car il oublie l'antecedan.

## 2- Inconvénients d’un RNN simple

#### 1- il ne pas pas appredre des relations complexe de text , car il utilise seulement une seule couche 

#### 2- il a un memoire courte

#### 3-peut pas generaliser , surtout qu'on a seulement 6 phrases , donc les nouvelles mots comme heureux et triste sont inconnue pour lui

## 3-Que se passerait-il si on avait des phrases beaucoup plus longues ?

#### - notre RNN va connaitre plus de mots 

#### probleme de gradient : notre rnn simple et peux pas apprendre des relations complexes , donc on doit utiliser des extensions comme LSTM ou GRU pour resoudre ca

#### - on va avoir un entrainement lent , car on va traite plus de sequences

#### il a perdre de precesion , cause il va trouver un difficulter de compredre le sens global 

#### solution : utlisation de plusieurs couche et des extensions du RNN , avec des robusts etape de pre-traitement de donnees.